In [42]:
from pytube import YouTube, Playlist, extract
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
from preprocessing import Utilities
from tqdm.notebook import tqdm
import os

In [43]:
ji = 'https://www.youtube.com/playlist?list=PLrFazGK5nFPwlVrpZkJMJa6wYDWgOg2xe'
huberman = 'https://www.youtube.com/playlist?list=PLPNW_gerXa4Pc8S2qoUQc5e8Ir97RLuVW'

In [93]:
playlist = Playlist(url=huberman)

In [89]:
playlist[0]

'https://www.youtube.com/watch?v=Sit1reDPBcY'

In [94]:
huberman_dir = './data/videos/huberman/'

In [87]:
def download_audio(video_url: str, out_dir: str):
  """Download youtube video and return videoDetails"""
  if not os.path.exists(out_dir):
      os.makedirs(out_dir)
  yt = YouTube(video_url)
  (yt.streams
    .filter(only_audio=True, file_extension = "mp4")
    .desc()
    .first()
    .download(filename = f"{os.path.join(out_dir, yt.video_id)}.mp4"))
  metadata = yt.vid_info['videoDetails']
  metadata['show_url'] = video_url
  return metadata

In [90]:
info = download_audio(playlist[0], huberman_dir)

In [95]:
meta_list= []

for num in range(3):
    meta = download_audio(playlist[num], out_dir=huberman_dir)
    meta_list.append(meta)

In [97]:
with open('./data/huberman_meta.json', 'w') as f:
    json.dump(meta_list, f)

In [ ]:
def create_video_url(video_id: str, playlist_id: str):
    return f'https://www.youtube.com/watch?v={video_id}&list={playlist_id}'

In [10]:
def get_meta_download_audio(link: str, download_audio: bool=False, audio_dest: str='./'):
    try:
        video = YouTube(link)
        meta = build_metadata_dict(video)
        if download_audio:
            audio = video.streams.filter(only_audio=True, file_extension='mp4').first()
            audio.download(audio_dest, skip_existing=False)
        return meta
    except Exception as e:
        print(e)

In [11]:
test = [get_meta_download_audio(link) for link in links]

In [12]:
len(test)

5

In [ ]:
def build_metadata_dict(ytobject: YouTube):
    meta = {}
    meta['video_id'] = ytobject.video_id
    meta['description'] = ytobject.description if ytobject.description else "No description provided" 
    meta['keywords'] = ytobject.keywords
    meta['length'] = ytobject.length
    meta['publish_date'] = ytobject.publish_date
    meta['thumbnail_url'] = ytobject.thumbnail_url
    meta['title'] = ytobject.title.strip()
    meta['views'] = ytobject.views
    return meta